### Coleta usando a API do twitter

In [20]:
#Coleta
import oauth2
print(oauth2.Consumer)

<class 'oauth2.Consumer'>


In [21]:
import json
import pandas as pd
import csv
#Pre processamento
import string 
import re
#Ngrams
import nltk
from nltk.tokenize import TweetTokenizer

In [4]:
def criaJsonSaida(json_input):
    return {
        "id": json_input["id"],
        "data_criacao": json_input["created_at"],
        "text": json_input["full_text"].lower(),
        "usuario": json_input["user"]["name"],
        "e_sobre_a_empresa": None
    }

def get_requisicao(cliente, max_id, termo):
    requisicao = cliente.request(
        'https://api.twitter.com/1.1/search/tweets.json?q={termo}&tweet_mode=extended&result_type=recent&lang=pt-br&count=3&{max_id}'.format(max_id=max_id, termo=termo))
    decodificar = requisicao[1].decode()
    try:
        objeto = json.loads(decodificar)
        return objeto['statuses']
    except Exception:
        print(objeto)

def coleta(api_key, api_secret_key, token_key, token_secret_key, termo, num_tweets, arquivo_saida):
    consumer = oauth2.Consumer(api_key, api_secret_key)
    token = oauth2.Token(token_key, token_secret_key)
    cliente = oauth2.Client(consumer, token)
    strUrl = ""
    max_id = ""
    num_tweets_collected = 0

    tweetsColetados = {
        "tweets": []
    }

    mantem_tweeets_nao_repetidos = set()
    
    try:
        # coleta os tweets
        while(num_tweets_collected < num_tweets):
            tweets = get_requisicao(cliente, max_id, termo)            
            for tweet in tweets:
                tweet_saida = criaJsonSaida(tweet)
                par_unico = (tweet_saida["usuario"], tweet_saida["data_criacao"])
                if par_unico not in mantem_tweeets_nao_repetidos:
                    mantem_tweeets_nao_repetidos.add(par_unico)
                    tweetsColetados["tweets"] += [criaJsonSaida(tweet)]
                
                max_id = "max_id={max_id}&".format(max_id=tweet['id'])
                num_tweets_collected += 1
    except Exception as e:
        print(e)
    finally:
        with open(arquivo_saida, 'w') as f:
            f.write(json.dumps(tweetsColetados["tweets"], indent=2))


In [5]:
# inicializa keys
api_key = 'suaAPIKey'
api_secret_key = 'suaAPISecretKey'
token_key = 'suaTokenKey'
token_secret_key = 'suaTokenSecretKey'
termo = 'sadia'
num_tweets = 100
arquivo_com_tweetes_coletados = 'tweets_coletados.json'

try:
    coleta(api_key, api_secret_key, token_key,
           token_secret_key, termo, num_tweets, arquivo_com_tweetes_coletados)
except Exception as e:
    print(e)

### Gerando csv com os tweets a serem rotulados

In [8]:
with open(arquivo_com_tweetes_coletados, 'r') as f:
    dic_tweets = json.loads(f.read())
    tweetsDF = pd.read_json(json.dumps(dic_tweets))
nome_arquivo_com_tweetes_rotulados = 'tweets_coletados.csv'
tweetsDF.to_csv(nome_arquivo_com_tweetes_rotulados, quoting=csv.QUOTE_ALL, index=False)

### Lendo o csv com os tweets rotulados

In [9]:
tweets_rotuladosDF = pd.read_csv(nome_arquivo_com_tweetes_rotulados,quoting=csv.QUOTE_ALL)

Quantidade de tweets por classe

In [11]:
sobre_a_sadia_qtd = tweets_rotuladosDF[tweets_rotuladosDF.e_sobre_a_empresa == 1]['e_sobre_a_empresa'].count()
sobre_outros_qtd = tweets_rotuladosDF['e_sobre_a_empresa'].count() - sobre_a_sadia_qtd
print("Sobre a Sadia: " +str(sobre_a_sadia_qtd) + " tweets")
print("Sobre outro assunto: " +str(sobre_outros_qtd) + " tweets")

Sobre a Sadia: 0 tweets
Sobre outro assunto: 0 tweets


### Identificando os brigramas e trigramas encontrados nos twittes

In [28]:
def novo_n_gram(ngram, bioutrigram, e_sobre_a_empresa):
    if e_sobre_a_empresa:
        ngram[bioutrigram] = {"sobre_a_sadia" : 1 , "sobre_outros" : 0}
    else:
        ngram[bioutrigram] = {"sobre_a_sadia" : 0 , "sobre_outros" : 1}
def incrementa_n_gram(ngram, bioutrigram, e_sobre_a_empresa):
    valor = ngram.get(bioutrigram)
    if e_sobre_a_empresa:
        valor["sobre_a_sadia"] += 1
    else:
        valor["sobre_outros"] += 1

In [29]:
tknzr = TweetTokenizer()

ngram = {}

for indice, linha in tweets_rotuladosDF.iterrows() :
    tokens = tknzr.tokenize(linha[tweets_rotuladosDF.text.name])
    for count in range(len(tokens)):
        if(tokens[count] == 'sadia'):
            bigram = (tokens[count - 1], tokens[count])    
            if ngram.get(bigram) == None:
                novo_n_gram(ngram, bigram, linha['e_sobre_a_empresa'])
            else:
                incrementa_n_gram(ngram, bigram, linha['e_sobre_a_empresa'])
            
            trigram = (tokens[count - 2], tokens[count - 1], tokens[count])
            if ngram.get(trigram) == None:
                novo_n_gram(ngram, trigram, linha['e_sobre_a_empresa'])
            else:
                incrementa_n_gram(ngram, trigram, linha['e_sobre_a_empresa'])         

ngram.items() retorna uma lista de tuplas com o seguinte formato ( ngram, {'sobre_a_sadia': qtd, 'sobre_outros': qtd} ), onde a posicao 0 é o ngram e a posicao 1 é o dicionário 


In [30]:
ngram_ordenado = sorted(ngram.items(), key=lambda x: x[1]['sobre_a_sadia'], reverse=True)

Imprimindo os n mais frequetes

In [31]:
n = 10
resultado = pd.DataFrame()
contador = 1
for tupla in ngram_ordenado:
    if(contador > n):
        break
    chaves = tupla[0]
    valores = tupla[1]
    total = valores['sobre_a_sadia'] + valores['sobre_outros']
    resultado = resultado.append({'ngram': chaves, 'sobre a sadia (%)': valores['sobre_a_sadia']/total, 'sobre outros (%)': valores['sobre_outros']/total}, ignore_index=True )
    contador += 1 
resultado

,ngram,sobre a sadia (%),sobre outros (%)
0,"(da, sadia)",1.0,0.0
1,"(mascote, da, sadia)",1.0,0.0
2,"(amizade, sadia)",1.0,0.0
3,"(brahma, sadia)",1.0,0.0
4,"(aquela, brahma, sadia)",1.0,0.0
5,"(ser, sadia)",1.0,0.0
6,"(:, ser, sadia)",1.0,0.0
7,"(forma, sadia)",1.0,0.0
8,"(de, forma, sadia)",1.0,0.0
9,"(pela, sadia)",1.0,0.0
